<center>
<h3>Agents Example(creating custom agents)</h3>
</center>

## Agents

<p>By themselves language model can't take actions- they just ouput text</p>
<p>Agents are system that use LLM as a reasoning engine to determine which action to take(means which tool should be called.) and what the input to those actions should be. The result of those action can be feed back into the agent and it determine whether more actions are needed or whether it is okay to finish.</p>

<p><code>LangGraph</code> is used to create highly controllable and cutomizable agents</p>

#### <font style="color:green">Simple agent  implementation using LangGraph</font>

In [18]:
from langgraph.graph import START,END,StateGraph #for graph
from langchain_core.tools import BaseTool # for tools 
from pydantic import BaseModel,Field # for type
from typing import Optional,Type,Literal,TypedDict
from langchain_core.callbacks import CallbackManagerForToolRun,AsyncCallbackManagerForToolRun



In [17]:
class BaseInputType(BaseModel):
    a : int = Field("First Number")
    b: int = Field("Second Number")

class BaseOutputType(BaseModel):
    c : int = Field("Result")
    
# Tool Implementation

# Note: It's important that every field has type hints. BaseTool is a
# Pydantic class and not having type hints can lead to unexpected behavior.

class SimpleCalculator(BaseTool):
    name : str = "Simple Calculator"
    description : str = "Helpful for simple math question"
    args_schema: Type[BaseModel] = BaseInputType
    return_direct:bool = True
    
    
    # for normal calls
    def _run(self,a : int,b:int,run_manager:Optional[CallbackManagerForToolRun] = None) -> BaseOutputType:
        return a*b
    
    # If the calculation is cheap, you can just delegate to the sync implementation
    # as shown below.
    # If the sync calculation is expensive, you should delete the entire _arun method.
    # LangChain will automatically provide a better implementation that will
    # kick off the task in a thread to make sure it doesn't block other async code. 
        
    # for async calls
    async def _arun(self,a:int,b:int, run_manager:Optional[AsyncCallbackManagerForToolRun] = None)->BaseOutputType:
        return self._run(a,b,run_manager)

# checking the tool
multiply = SimpleCalculator()

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args_schema)

Simple Calculator
Helpful for simple math question
<class '__main__.BaseInputType'>


In [15]:
print(multiply.invoke({"a":2,"b":3}))
print(await multiply.ainvoke({"a":25,"b":25}))

6
625


### Agent(using LangGraph)

In [29]:
# Type of state
class State(TypedDict):
    first_num : int
    sec_num : int
    result : int

# Defining the functionality of Node
def node_1(state):
    # let's update the state in node 1
    print("--- Inside Node 1 (Updating the state) ---- \n")
    print("State : ",state,"\n")
    return ({"first_num" : state["first_num"], "sec_num":state["sec_num"]})

def node_2(state):
    print("---- Inside Node 2 (returning multiply) ----\n")
    return({"result":state["first_num"] * state["sec_num"]})

def node_3(state):
    print("---- Inside node 3 (returning addition) ----\n")
    return({"result" : state["first_num"] + state["sec_num"]})

def deciding_node(state)-> Literal["node_2","node_3"]:
    if state["first_num"] < state["sec_num"]:
        return "node_2"
    return "node_3"

# Creating Graph
builder = StateGraph(State)

# Adding node to graph
builder.add_node("node_1",node_1)
builder.add_node("node_2",node_2)
builder.add_node("node_3",node_3)

# Adding edge between graph
builder.add_edge(START,"node_1")
builder.add_conditional_edges("node_1",deciding_node)
builder.add_edge("node_2",END)
builder.add_edge("node_3",END)

# at last compiling the graph
graph = builder.compile()

In [30]:
graph.invoke({"first_num":10,"sec_num":30})

--- Inside Node 1 (Updating the state) ---- 

State :  {'first_num': 10, 'sec_num': 30} 

---- Inside Node 2 (returning multiply) ----



{'first_num': 10, 'sec_num': 30, 'result': 300}

In [31]:
graph.invoke({"first_num":100,"sec_num":30})

--- Inside Node 1 (Updating the state) ---- 

State :  {'first_num': 100, 'sec_num': 30} 

---- Inside node 3 (returning addition) ----



{'first_num': 100, 'sec_num': 30, 'result': 130}

#### Exmaple of 4 key concept using agent
<ul>
<li>Using <code>Chat message</code> in our graph</li>
<li>using <code>Chat Models </code></li>
<li><code>Binding tools</code> to LLM</li>
<li>Executing <code>toolcall</code> in graph</li>
</ul>

#### Message
--desc-- chain timestamp : 0.31

In [13]:
from pprint import pprint
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
messages = [AIMessage(content="So you said you were searching ocean manmals?",role="model",name="model")]
messages.extend([HumanMessage(content="Yes that's right.", role="user",name="user")])
messages.extend([AIMessage(content="Great, what would you like to learn about", role="model",name="model")])
messages.extend([HumanMessage(content="I want to learn about the whale in ocean",role="user",name="user")])

for m in messages:
    print(m.pretty_print())

================================== Ai Message ==================================
Name: model

So you said you were searching ocean manmals?
None
================================ Human Message =================================
Name: user

Yes that's right.
None
================================== Ai Message ==================================
Name: model

Great, what would you like to learn about
None
================================ Human Message =================================
Name: user

I want to learn about the whale in ocean
None


#### Chat Models
--desc 1.03

In [16]:
from langchain_community.llms.ollama import Ollama
llm = Ollama(model="llama3.1")
result = llm.invoke(messages)
type(result)

str

In [18]:
print(result)

There are so many fascinating things about whales.

Let's dive into it (pun intended). Whales are massive marine mammals that belong to the order Cetacea. There are two suborders: toothed whales (Odontoceti) and baleen whales (Mysticeti).

Toothed whales include dolphins, porpoises, and sperm whales. They have a highly developed sense of hearing and echolocation, which helps them navigate and hunt in the dark depths of the ocean.

Baleen whales, on the other hand, are filter feeders that use their baleen plates to strain tiny crustaceans, plankton, and small fish from the water. The largest animal to have ever existed on Earth is a blue whale, which can grow up to 100 feet (30 meters) in length and weigh around 200 tons!

Some interesting facts about whales include:

* Whales are highly social creatures that often live in groups called pods.
* They are incredibly intelligent animals that have been observed using complex behaviors like cooperative hunting and even playing.
* Some specie

In [21]:
# Implement tool yo give below capability TODO

In [20]:
print(llm.invoke([HumanMessage(content="what's the day today",role="user")]))

I'm happy to help, but I don't have real-time access to your current date and time. However, I can suggest a few options:

1. **Check your phone or computer**: Take a look at the clock on your device to see the current date.
2. **Use an online calendar**: Websites like Google Calendar, Apple Calendar, or any other digital calendar you might use should be able to tell you the day of the week and date.
3. **Ask me about general information**: If you're looking for something specific (e.g., today's weather forecast), feel free to ask!

Which option sounds most convenient for you?


#### Tools
---dec 1.32

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# API keys and Lang Smith 
import os
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")


In [17]:
from langchain_core.tools import Tool
from langchain_core.tools import tool

@tool
def multiply(a : int,b:int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a*b

# Bind the tool to the LLM
llm_with_tool = llm.bind_tools([multiply])

In [18]:
query = "What is 3 * 12?"

ai_message = llm_with_tool.invoke(query)

In [19]:
print(ai_message)

content='' additional_kwargs={'tool_calls': [{'id': 'call_XysdXilX82HhY4Tsg6mCnC78', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 65, 'total_tokens': 82, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-1ac203e1-161c-46b9-8435-da4322c61f88-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_XysdXilX82HhY4Tsg6mCnC78', 'type': 'tool_call'}] usage_metadata={'input_tokens': 65, 'output_tokens': 17, 'total_tokens': 82}


In [20]:
ai_msg = ai_message
ai_msg.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 12},
 'id': 'call_XysdXilX82HhY4Tsg6mCnC78',
 'type': 'tool_call'}

In [31]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XysdXilX82HhY4Tsg6mCnC78', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 65, 'total_tokens': 82, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1ac203e1-161c-46b9-8435-da4322c61f88-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_XysdXilX82HhY4Tsg6mCnC78', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 17, 'total_tokens': 82})

In [21]:
{"multiply": multiply}

{'multiply': StructuredTool(name='multiply', description='Add two integers.\n\n    Args:\n        a: First integer\n        b: Second integer', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x11900f9c0>)}

In [22]:
[ai_msg.tool_calls[0]['name'].lower()]

['multiply']

In [23]:
{"multiply": multiply}[ai_msg.tool_calls[0]['name'].lower()]

StructuredTool(name='multiply', description='Add two integers.\n\n    Args:\n        a: First integer\n        b: Second integer', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x11900f9c0>)

In [26]:
tool_message = {"multiply": multiply}[ai_msg.tool_calls[0]['name'].lower()].invoke(ai_msg.tool_calls[0])
tool_message

ToolMessage(content='36', name='multiply', tool_call_id='call_XysdXilX82HhY4Tsg6mCnC78')

In [32]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12?', role='user'),
 ToolMessage(content='36', name='multiply', tool_call_id='call_XysdXilX82HhY4Tsg6mCnC78'),
 ToolMessage(content='36', name='multiply', tool_call_id='call_XysdXilX82HhY4Tsg6mCnC78')]

In [33]:
llm_with_tool.invoke(messages)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[1].role', 'code': None}}

In [30]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content=query,role='user')]
messages.append(tool_message)
llm_with_tool.invoke(messages)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[1].role', 'code': None}}